### Import

In [67]:
# Import the NumPy library for matrix manipulation
import numpy as np

# Simplified DES

In [68]:
P10 = [3, 5, 2, 7, 4, 10, 1, 9, 8, 6]
P8 = [6, 3, 7, 4, 8, 5, 10, 9]
IP = [2, 6, 3, 1, 4, 8, 5, 7]
IP_inv = [4, 1, 3, 5, 7, 2, 8, 6]
E_P = [4, 1, 2, 3, 2, 3, 4, 1]
P4 = [2, 4, 3, 1]
S0 = [[1, 0, 3, 2],
    [3, 2, 1, 0],
    [0, 2, 1, 3],
    [3, 1, 3, 2]]
S1 = [[0, 1, 2, 3],
    [2, 0, 1, 3],
    [3, 0, 1, 0],
    [2, 1, 0, 3]]

def apply_permutation(bits, permutation):
    """Applies a given permutation to the input bits."""
    permuted_bits = ""
    for idx in permutation:
        permuted_bits += bits[idx - 1]
    return permuted_bits

def rotate_left(bits, num_shifts):
    """Performs a circular left shift on the input bits."""
    return bits[num_shifts:] + bits[:num_shifts]

def create_subkeys(initial_key):
    """Generates the subkeys (K1 and K2) from the 10-bit initial key."""
    print(f"Initial 10-bit Key: {initial_key}")
    
    # Apply P10 permutation
    initial_key = apply_permutation(initial_key, P10)
    print(f"After P10 Permutation: {initial_key}")
    
    # Left rotate by 1
    initial_key = rotate_left(initial_key, 1)
    print(f"After Left Rotation by 1: {initial_key}")
    
    # Generate Subkey 1 (K1) using P8
    subkey1 = apply_permutation(initial_key, P8)
    print(f"Subkey 1 (K1) after P8 Permutation: {subkey1}")
    
    # Left rotate by 2
    initial_key = rotate_left(initial_key, 2)
    print(f"After Left Rotation by 2: {initial_key}")
    
    # Generate Subkey 2 (K2) using P8
    subkey2 = apply_permutation(initial_key, P8)
    print(f"Subkey 2 (K2) after P8 Permutation: {subkey2}")
    
    return subkey1, subkey2


def fk_function(data, subkey):
    """The function f_k used in the Simplified DES algorithm."""
    expanded_data = apply_permutation(data, E_P)
    xor_result = xor_bits(expanded_data, subkey)
    row_0 = int(xor_result[0] + xor_result[3], 2)
    col_0 = int(xor_result[1] + xor_result[2], 2)
    row_1 = int(xor_result[4] + xor_result[7], 2)
    col_1 = int(xor_result[5] + xor_result[6], 2)
    sbox_output = bin(S0[row_0][col_0])[2:].zfill(2) + bin(S1[row_1][col_1])[2:].zfill(2)
    return apply_permutation(sbox_output, P4)

def xor_bits(bits_a, bits_b):
    """Performs a bitwise XOR operation on two bit strings."""
    xor_result = ""
    for i in range(len(bits_a)):
        xor_result += "0" if bits_a[i] == bits_b[i] else "1"
    return xor_result

def sdes_encrypt_block(block, key):
    """Encrypts an 8-bit block using the Simplified DES algorithm."""
    print(f"Initial Block: {block}")
    subkey1, subkey2 = create_subkeys(key)
    print(f"Subkey 1: {subkey1}, Subkey 2: {subkey2}")
    
    data = apply_permutation(block, IP)
    print(f"After Initial Permutation (IP): {data}")
    
    left_half = data[:4]
    right_half = data[4:]
    fk_result = fk_function(right_half, subkey1)
    data = xor_bits(left_half, fk_result) + right_half
    print(f"After Round 1 (fk with subkey1): {data}")
    
    data = data[4:] + data[:4]  # Swap the halves
    print(f"After Swap: {data}")
    
    left_half = data[:4]
    right_half = data[4:]
    fk_result = fk_function(right_half, subkey2)
    data = xor_bits(left_half, fk_result) + right_half
    print(f"After Round 2 (fk with subkey2): {data}")
    
    encrypted_block = apply_permutation(data, IP_inv)
    print(f"After Final Permutation (IP_inv): {encrypted_block}")
    
    return encrypted_block

def sdes_decrypt_block(block, key):
    """Decrypts an 8-bit block using the Simplified DES algorithm."""
    print(f"Initial Block: {block}")
    subkey1, subkey2 = create_subkeys(key)
    print(f"Subkey 1: {subkey1}, Subkey 2: {subkey2}")
    
    data = apply_permutation(block, IP)
    print(f"After Initial Permutation (IP): {data}")
    
    left_half = data[:4]
    right_half = data[4:]
    fk_result = fk_function(right_half, subkey2)
    data = xor_bits(left_half, fk_result) + right_half
    print(f"After Round 1 (fk with subkey2): {data}")
    
    data = data[4:] + data[:4]  # Swap the halves
    print(f"After Swap: {data}")
    
    left_half = data[:4]
    right_half = data[4:]
    fk_result = fk_function(right_half, subkey1)
    data = xor_bits(left_half, fk_result) + right_half
    print(f"After Round 2 (fk with subkey1): {data}")
    
    decrypted_block = apply_permutation(data, IP_inv)
    print(f"After Final Permutation (IP_inv): {decrypted_block}")
    
    return decrypted_block


# Enhanced S-DES
This section contains the necessary functions for the transformations and shifting prior to applying the S-DES algorithm.


In [69]:
# Define the transposition key and calculate the number of columns
transp_key = "emirube"
n_columns = len(transp_key)
n_rows = 0 # Initially, the number of rows is unknown

# Function to reorder the columns of a matrix based on a given order
def MR(arr, order):
    # Initialize an empty string to hold the resulting text
    result_string = ""
    
    # Iterate through the columns based on the provided order
    for col in order:
        # Extract the entire column and append the values row by row
        for row in arr[:, col]:
            result_string += row  # Concatenate each element
    
    return result_string 

# Function to revert the column reordering of a matrix
def revert_MR(arr, order):
    # Step 1: Convert the array to a string by concatenating all values row by row
    text = ''.join(''.join(row) for row in arr)
    
    # Initialize a result array (same shape as the original)
    n_rows, n_columns = arr.shape
    result_matrix = np.full((n_rows, n_columns), '', dtype='<U1')  # Create empty matrix to store result
    
    # Step 2: Fill the result matrix column by column, based on the 'order'
    idx = 0  # Index to keep track of characters in the string
    for col in order:
        for row in range(n_rows):
            if arr[row, col] != '':  # Only fill in non-empty positions
                if idx < len(text):  # Make sure we don't go out of bounds
                    result_matrix[row, col] = text[idx]
                    idx += 1
    
    # Step 3: Convert the result matrix back into a single string
    result_string = ''.join(''.join(row) for row in result_matrix)
    
    return result_string

# Function to create a matrix based on the provided text
def matrixmaker(text):
    # Initialize the matrix with empty strings
    matrix = np.full((n_rows, n_columns), '', dtype='<U1')

    # Fill the matrix row by row
    for i in range(n_rows):
        # Slice the text list to get the next row
        start_idx = i * n_columns
        end_idx = start_idx + n_columns
        row = text[start_idx:end_idx]  # Get the characters for the current row
        
        # Assign the sliced row to the matrix
        matrix[i, :len(row)] = list(row)  # Only fill as many columns as the row has characters
    
    return matrix

def shift_rows_left(matrix):
    n_rows, n_columns = matrix.shape
    
    # Create a new matrix to store the result
    shifted_matrix = np.empty_like(matrix)
    
    # Perform the row shifting to the left (ignoring empty strings '')
    for i in range(n_rows):
        # Get the non-empty values in the row
        non_empty_values = [val for val in matrix[i] if val != '']
        
        # Perform the circular shift on the non-empty values
        shift = i % len(non_empty_values) if len(non_empty_values) > 0 else 0
        
        # Roll (shift) the non-empty values to the left
        shifted_non_empty_values = np.roll(non_empty_values, -shift)
        
        # Fill the shifted matrix, preserving positions of empty ('') values
        idx = 0  # Index for non-empty values
        for j in range(n_columns):
            if matrix[i, j] != '':
                shifted_matrix[i, j] = shifted_non_empty_values[idx]
                idx += 1
            else:
                shifted_matrix[i, j] = ''  # Leave the empty string as is
    
    return shifted_matrix

def shift_rows_right(matrix):
    n_rows, n_columns = matrix.shape
    
    # Create a new matrix to store the result
    reverted_matrix = np.empty_like(matrix)
    
    # Perform the reverse row shifting (circular shift, ignoring '')
    for i in range(n_rows):
        # Get the non-empty values in the row
        non_empty_values = [val for val in matrix[i] if val != '']
        
        # Perform the circular shift on the non-empty values
        shift = i % len(non_empty_values) if len(non_empty_values) > 0 else 0
        
        # Roll (shift) the non-empty values to the right
        shifted_non_empty_values = np.roll(non_empty_values, shift)
        
        # Fill the reverted matrix, preserving positions of empty ('') values
        idx = 0  # Index for non-empty values
        for j in range(n_columns):
            if matrix[i, j] != '':
                reverted_matrix[i, j] = shifted_non_empty_values[idx]
                idx += 1
            else:
                reverted_matrix[i, j] = ''  # Leave the empty string as is
    
    return reverted_matrix

## Example of Enhanced S-DES

In [70]:
# Key for encryption and decyption of S-DES
key = "1010000010"
# Random list that defines the column order
random_list = [3, 4, 5, 6, 2, 0, 1]
print(random_list)

# plaintext = input("Enter plaintext:")
plaintext = "are you coming to the toga party"
plaintext = plaintext.replace(" ", "")

ptcode = [char for char in plaintext]

[3, 4, 5, 6, 2, 0, 1]


In [71]:
# Calculate the number of rows needed based on the pre-shared number of columns and text
n_rows = (len(ptcode) + n_columns - 1) // n_columns  # Ceiling division

ogmatrix = matrixmaker(ptcode)
print(ogmatrix)
print()
firstround = MR(ogmatrix, random_list)
print("First round:", firstround)

fstmatrix = matrixmaker(firstround)
print(fstmatrix)
print()
secondround = MR(fstmatrix, random_list)
print("Second round:", secondround)

trans_output = matrixmaker(secondround)
print("Transposition output:")
print(trans_output)

[['a' 'r' 'e' 'y' 'o' 'u' 'c']
 ['o' 'm' 'i' 'n' 'g' 't' 'o']
 ['t' 'h' 'e' 't' 'o' 'g' 'a']
 ['p' 'a' 'r' 't' 'y' '' '']]

First round: ynttogoyutgcoaeieraotprmha
[['y' 'n' 't' 't' 'o' 'g' 'o']
 ['y' 'u' 't' 'g' 'c' 'o' 'a']
 ['e' 'i' 'e' 'r' 'a' 'o' 't']
 ['p' 'r' 'm' 'h' 'a' '' '']]

Second round: tgrhocaagoooatttemyyepnuir
Transposition output:
[['t' 'g' 'r' 'h' 'o' 'c' 'a']
 ['a' 'g' 'o' 'o' 'o' 'a' 't']
 ['t' 't' 'e' 'm' 'y' 'y' 'e']
 ['p' 'n' 'u' 'i' 'r' '' '']]


In [72]:
shifted_output = shift_rows_left(trans_output)
print("Shifted output:")
print(shifted_output)

Shifted output:
[['t' 'g' 'r' 'h' 'o' 'c' 'a']
 ['g' 'o' 'o' 'o' 'a' 't' 'a']
 ['e' 'm' 'y' 'y' 'e' 't' 't']
 ['i' 'r' 'p' 'n' 'u' '' '']]


In [73]:
# Flatten the shifted matrix into a long list
flattened_list = shifted_output.flatten().tolist()

# Remove all '' values
filtered_list = [val for val in flattened_list if val != '']

# Print the flattened list
print(filtered_list)

binary_list = [format(ord(char), '08b') for char in filtered_list]
print(binary_list)

['t', 'g', 'r', 'h', 'o', 'c', 'a', 'g', 'o', 'o', 'o', 'a', 't', 'a', 'e', 'm', 'y', 'y', 'e', 't', 't', 'i', 'r', 'p', 'n', 'u']
['01110100', '01100111', '01110010', '01101000', '01101111', '01100011', '01100001', '01100111', '01101111', '01101111', '01101111', '01100001', '01110100', '01100001', '01100101', '01101101', '01111001', '01111001', '01100101', '01110100', '01110100', '01101001', '01110010', '01110000', '01101110', '01110101']


In [74]:
# List to store the decrypted binary numbers
decrypted_list = []

# Encrypt and decrypt each binary number in the list
for binary in binary_list:
    ciphertext = sdes_encrypt_block(binary, key)
    decrypted_text = sdes_decrypt_block(ciphertext, key)
    
    # Append the decrypted binary string to the list
    decrypted_list.append(decrypted_text)
    
    # Print each step for verification
    print(f"Plaintext: {binary}")
    print(f"Ciphertext: {ciphertext}")
    print(f"Decrypted Text: {decrypted_text}")
    print("-" * 30)  # Separator for better readability

Initial Block: 01110100
Initial 10-bit Key: 1010000010
After P10 Permutation: 1000001100
After Left Rotation by 1: 0000011001
Subkey 1 (K1) after P8 Permutation: 10100010
After Left Rotation by 2: 0001100100
Subkey 2 (K2) after P8 Permutation: 00011100
Subkey 1: 10100010, Subkey 2: 00011100
After Initial Permutation (IP): 11101000
After Round 1 (fk with subkey1): 01111000
After Swap: 10000111
After Round 2 (fk with subkey2): 11010111
After Final Permutation (IP_inv): 11001111
Initial Block: 11001111
Initial 10-bit Key: 1010000010
After P10 Permutation: 1000001100
After Left Rotation by 1: 0000011001
Subkey 1 (K1) after P8 Permutation: 10100010
After Left Rotation by 2: 0001100100
Subkey 2 (K2) after P8 Permutation: 00011100
Subkey 1: 10100010, Subkey 2: 00011100
After Initial Permutation (IP): 11010111
After Round 1 (fk with subkey2): 10000111
After Swap: 01111000
After Round 2 (fk with subkey1): 11101000
After Final Permutation (IP_inv): 01110100
Plaintext: 01110100
Ciphertext: 110011

In [75]:
# Now convert the binary values in decrypted_list to characters
decrypted_message = ''.join([chr(int(binary, 2)) for binary in decrypted_list])

# Print the final decrypted message
print("Decrypted Message:", decrypted_message)
SDES_ouput = matrixmaker(decrypted_message)

Decrypted Message: tgrhocagoooataemyyettirpnu


In [76]:
reverted_output = shift_rows_right(SDES_ouput)
print("Reverted output:")
print(reverted_output)

Reverted output:
[['t' 'g' 'r' 'h' 'o' 'c' 'a']
 ['a' 'g' 'o' 'o' 'o' 'a' 't']
 ['t' 't' 'e' 'm' 'y' 'y' 'e']
 ['p' 'n' 'u' 'i' 'r' '' '']]


In [77]:
rev_round = revert_MR(reverted_output, random_list)
print(rev_round)
rev1 = matrixmaker(rev_round)
print(rev1)
print()

rev_round2 = revert_MR(rev1, random_list)
print(rev_round2)
rev2 = matrixmaker(rev_round2)
print(rev2)

ynttogoyutgcoaeieraotprmha
[['y' 'n' 't' 't' 'o' 'g' 'o']
 ['y' 'u' 't' 'g' 'c' 'o' 'a']
 ['e' 'i' 'e' 'r' 'a' 'o' 't']
 ['p' 'r' 'm' 'h' 'a' '' '']]

areyoucomingtothetogaparty
[['a' 'r' 'e' 'y' 'o' 'u' 'c']
 ['o' 'm' 'i' 'n' 'g' 't' 'o']
 ['t' 'h' 'e' 't' 'o' 'g' 'a']
 ['p' 'a' 'r' 't' 'y' '' '']]
